# Big Data & Streaming: Exercise Results

## 1. Working with PySpark DataFrames
- Initialize a SparkSession, load a CSV file as a DataFrame, and display the inferred schema to understand the data structure.


In [ ]:
## 1. Working with PySpark DataFrames
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("PySparkDataFrameExample").getOrCreate()

# Load CSV file as DataFrame (inferring schema)
df = spark.read.csv('data.csv', header=True, inferSchema=True)

# Display inferred schema to understand the data structure
df.printSchema()

## 2. Advanced Filtering and Aggregation

- Using PySpark, filter the DataFrame to include only rows where the `value` column exceeds 100.
- Then, group the filtered data by the `category` column and compute the count of rows for each group.
- Display the resulting grouped counts.


In [ ]:
# Filter rows where 'value' > 100, group by 'category', and count rows per group
filtered_df = df.filter(df['value'] > 100)
grouped_counts = filtered_df.groupBy('category').count()
grouped_counts.show()

## 3. Spark Transformations
- Practice chaining multiple DataFrame transformations in PySpark. For example: filter the data based on a condition, select a subset of columns, and then group the data by a categorical column to perform an aggregation. Finally, execute an action (such as `.show()` or `.collect()`) to trigger the computation and view the results.


In [ ]:
# Chain multiple DataFrame transformations in PySpark
# Example: filter rows where value > 100, select category and value, group by category and calculate average value, then show results

from pyspark.sql.functions import avg

result = (
    df.filter(df['value'] > 100)
      .select('category', 'value')
      .groupBy('category')
      .agg(avg('value').alias('avg_value'))
)

result.show()

## 4. Kafka Streaming
- Implement a Python script that demonstrates real-time messaging with Kafka:
    - Create a Kafka producer to send sample messages to a local Kafka topic.
    - Create a Kafka consumer to read and print messages from that topic.
    - Ensure your script uses `localhost` as the Kafka broker.
    - Use separate producer and consumer logic (can be in the same script or separate scripts).
    - Test end-to-end message flow by observing that sent messages are received and displayed by the consumer.


In [ ]:
from kafka import KafkaProducer, KafkaConsumer

# Create producer
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Send sample messages to topic
for i in range(10):
    producer.send('topic', value=f'Message {i}'.encode('utf-8'))

# Create consumer
consumer = KafkaConsumer('topic', bootstrap_servers='localhost:9092', auto_offset_reset='earliest')

# Read and print messages from topic
for msg in consumer:
    print(msg.value.decode('utf-8'))


---

### Challenge
- Simulate a real-time data stream of random numbers and process it using PySpark Structured Streaming.
- Your task: Continuously read the stream, compute and update the running average of the numbers in real time, and output the results as the stream progresses.
- This exercise will help you practice handling streaming data, applying windowed aggregations, and working with PySpark's structured streaming APIs.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

spark = SparkSession.builder.getOrCreate()

# Simulate a stream of random numbers using Spark's 'rate' source
stream_df = spark.readStream.format('rate').option('rowsPerSecond', 1).load()

# Compute the running average using aggregation over all rows seen so far
running_avg_df = stream_df.groupBy().agg(avg('value').alias('running_avg'))

# Output the updated running average to the console as the stream progresses
query = running_avg_df.writeStream.outputMode('complete').format('console').start()
query.awaitTermination()
